In [ ]:
!pip install --upgrade --quiet openai duckpy

In [ ]:
import os, openai
from google.colab import userdata
os.environ["OPENAI_API_KEY"]=userdata.get('op')
openai.api_key = os.getenv("OPENAI_API_KEY")\

In [ ]:
#이전 방식
client=openai.OpenAI()

content="""다음 문장에서 색 정보가 있으면 그것을 추출하여 영어로 번역하고 json 포맷으로 만들어서 출력해줘"
문장: \"나는 빨간 사과가 좋더라\n
"""
response=client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": content}
    ]
)
print(response)
#이걸 실행하면 결과가 실행 할때마다 달라짐 틀릴 때도 있
#그러면 원하는 데이터 형태를 못구함

In [ ]:
#이전 방식
client=openai.OpenAI()

content="""다음 문장에서 색 정보가 있으면 그것을 추출하여 영어로 번역하고 json 포맷으로 만들어서 출력해줘"
문장: \"아침에는 인간적으로 커피 한 잔 주고 뭐라 하자\n
"""
response=client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": content}
    ]
)
print(response)
#색 정보가 없는 것을 찾아야 하는데 못 찾고 번역도 이상하게 하는 모습 보여

In [ ]:
schema={
    "$schema": "http://json-schema.org/draft-07/schema#",
    "type": "object
    "properties": {
        "color": {
            "type": "string",
        }
    },
    "required": ["color"]
} #필요한 데이터 형태를 지정해줘버림

content=f"""다음 문장에서 색 정보가 있으면 그것을 추출하여 영어로 번역하고 json 포맷으로 만들어서 출력해줘.
문장: \"나는 빨간 사과가 좋더라\n
json schema는 다음과 같다 :{schema}
색깔 정보가 없으면 no color로 대신해
"""
#output을 아예 지정하는게 꿀팁 프롬프트 엔지니어링 할때 이 방법 추천함
response=client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": content}
    ]
)
print(response)
#얼추 답이 원하는 대로 나옴 근데 한글도 나옴 뭔가 아쉬움

잘못된 출력의 예:  
* 틀린 아웃풋(output 텍스트가 포함됨):output:{'color':'blue'}
* 틀린 아웃풋: 사용자는 노란색을 좋아합니다.
* 틀린 아웃풋: 주신 내용에서는 색 정보를 찾을 수가 없는데 더 도와드릴 일이 있나요?

In [ ]:
#새로운 방식

response = client.chat.completions.create(
    model="gpt-4"
    messages=[{"role":"user","content":"나는 노란색이 좋더라"}],
    functions=[ #이부분을 function calling 이라고 하는 듯? 정보를 추가 하는 느낌같음
        {
            "name":"color_extractor",
            "description":"사용자의 선호 색을 찾아서 영어로 번역하고 영어 단어를 출력함",
            "parameters": {
                "type":"object",
                "properties": {
                    "color": {
                        "type": "string",
                        "description":"색 정보를 빼내와서 그에 해당하는 영어 단어를 출력함"
                    },
                },
                "required": ["color"]
            },
        }
    ]
)

In [ ]:
print(response.choices[0].message)

In [ ]:
print(response.choices[0].message.function_call.arguments)
#function을 썻으니 그에 대한 데이터 얻으려면 이렇게 뽑아야 함

In [ ]:
#새로운 방식

response = client.chat.completions.create(
    model="gpt-4"
    messages=[{"role":"user","content":"퇴근했는데 업무 톡 날리는 양심없는 인간"}],
    functions=[ #이부분을 function calling 이라고 하는 듯? 정보를 추가 하는 느낌같음
        {
            "name":"color_extractor",
            "description":"사용자의 선호 색을 찾아서 영어로 번역하고 영어 단어를 출력함",
            "parameters": {
                "type":"object",
                "properties": {
                    "color": {
                        "type": "string",
                        "description":"색 정보를 빼내와서 그에 해당하는 영어 단어를 출력함"
                    },
                },
                "required": ["color"]
            },
        }
    ]
)
print(response.choices[0].message.function_call.arguments)
#색에 대한 정보 없을때 None을 반환

In [ ]:
#새로운 방식

response = client.chat.completions.create(
    model="gpt-4"
    messages=[{"role":"user","content":"아침에 세 개 저녁에 네개"}],
    functions=[ #이부분을 function calling 이라고 하는 듯? 정보를 추가 하는 느낌같음
        {
            "name":"apple_add",
            "description":"사과 숫자를 더함",
            "parameters": {
                "type":"object",
                "properties": {
                    "quantity1": {
                        "type": "integer",
                        "description":"첫번째 숫자"
                    },
                    "quantity2": {
                        "type": "integer",
                        "description":"두번째 숫자
                    },
                },
                "required": ["quantity1","quantity2"]
            },
        }
    ],
    function_call="auto"
)
print(response.choices[0].message.function_call.arguments)


In [ ]:
import json
from duckpy import Client
import ast

duckduckgo_client = Client()
def duck_search(query)->str:
  output= duckduckgo_client.search(query)
  return str(output)

completion = client.chat.completions.create(
    model="gpt-4",
    temperature=0, #완전 확률 높은거만 뽑는 듯
    functions=[ #이부분을 function calling 이라고 하는 듯? 정보를 추가 하는 느낌같음
        {
            "name":"duck_search",
            "description":"Used to search online",
            "parameters": {
                "type":"object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description":"Translate the Korean content into English input query"
                    },
                },
                "required": ["query"]
            },
        }
    ],
    function_call="auto"
    messages=[
        {"role":"system","content":"You must use the 'duck_search' function to search online."},
        {"role":"user","content":"조지 클루나 생일이 언제야?"}
        ]
)
print(completion)

#function_call이 뽑은 query는 "George Clooney birthday"가 나

In [ ]:
message= completion.choices[0].message
if message.function_call:
  function_name = message.function_call.name
  args= ast.literal_eval(message.function_call.arguments)
  function_response = duck_search(args["query"])
  print(function_response)
  completion_final=client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role":"user","content":"조지 클루나생일이 언제야?"},
        message,
        {
            "role":"function",
            "name":function_name,
            "content":function_response,
        },
    ]
)
print(completion_final.choices[0].message.content)

In [ ]:
example_user_input="런던 날씨가 안 좋다면서?"
completion= client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role":"user","content":example_user_input}
    ],
    functions=[
        {
            "name":"get_different_responses",
            "description":"답변을 여러가지 말투로 하기",
            "parameters": {
                "type":"object",
                "properties": {
                    "response_up": {
                        "type": "string",
                        "description":"답변을 높임말로 바꿈"
                    },
                    "response_down": {
                        "type": "string",
                        "description":"답변을 반말로 바꿈"
                    },
                },
                "required": ["response_up","response_down"]
            },
        }
    ],
    function_call={"name":"get_different_responses"}
)
reply_content=completion.choices[0].message
print(reply_content.function_call.arguments)
'''
답변
{
  "response_up": "네, 런던 날씨는 현재 좋지 않아요.",
  "response_down": "응 런던 날씨가 안 좋아."
}
'''

In [ ]:
example_user_input=\
'''레드헷 리눅스에서 새로운 사용자 만들고
그 사용자에게 /usr/arb 폴더 읽기 쓰기 사용권한을 주려면 어떻게 해야 해?
'''
completion= client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role":"user","content":example_user_input}
    ],
    functions=[
        {
            "name":"get_commands",
            "description":"Get a list of bash commands on an RedHat machine",
            "parameters": {
                "type":"object",
                "properties": {
                    "commands": {
                        "type": "array",
                        "items": {
                            "type": "string",
                            "description":"A terminal command string"
                        },
                        "description":"List of terminal command strings to execute"
                    }
                },
                "required": ["commands"]
            }
        },
    ],
    function_call={"name":"get_different_responses"}
)
reply_content=completion.choices[0].message
print(reply_content.function_call.arguments)

'''
{
  "commands": [
    "sudo adduser new_user",
    "sudo chown new_user /usr/arb",
    "sudo chmod 777 /usr/arb"
  ]
}
'''

In [ ]:
import ast
example_user_input="주말에 친구들 오는데 뭐 먹지? 소고기랑 감자랑 밀가루 있어"

def get_recipes_with_ingredients(ingredients):
  return f"{ingredients} 를 다 볶아서 볶음밥으로 만들어 먹읍시다!"
completion= client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role":"user","content":example_user_input}
    ],
    functions=[
        {
            "name":"get_recipes_with_ingredients",
            "description":"주어진 텍스트에서 음식을 만들 수 있는 재료 이름을 뽑아낸다.",
            "parameters": {
                "type":"object",
                "properties": {
                    "ingredients": {
                        "type": "string",
                        "description":"음식 재료"
                    },
                },
                "required": ["ingredients"]
            }
        },
    ],
    function_call={"name":"get_recipes_with_ingredients"}
)
reply_content=completion.choices[0].message
print(reply_content.function_call.arguments)

'''
{
  "commands": [
    "sudo adduser new_user",
    "sudo chown new_user /usr/arb",
    "sudo chmod 777 /usr/arb"
  ]
}
'''
